# TSFEDL Models

In [ ]:
import json
import os
import sys
sys.path.append('../')  ### to detect libraries in the parent directory
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from libraries.utils import *

import multiprocess
from multiprocess import Pool, cpu_count
from itertools import repeat
import time


pool = Pool(processes=(cpu_count() - 2))
print(cpu_count())


## Load data

In [ ]:
# ############ configuration - trace ################
# ############################################


CODE = 'mamba2'       ### application (code)       ###  'theft_protection', 'mamba2', 'lora_ducy'
BEHAVIOUR_FAULTY = 'faulty_data'            ### normal, faulty_data
BEHAVIOUR_NORMAL = 'normal'            ### normal, faulty_data
THREAD = 'single'           ### single, multi
VER = 4                     ### format of data collection

base_dir = '../../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
normalbase_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR_NORMAL}'
faultybase_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR_FAULTY}'

print(normalbase_path)
print(faultybase_path)


################# configuration - diag ################
IS_VAR_WINDOW = False             ### True: varibale window size, False: fixed window size; wether to use variable window size or not

#####################################################

train_base_path = os.path.join(normalbase_path, 'train_data')
ref_samples_basepath = os.path.join(normalbase_path, 'diag_refsamples')
# ref_var_samples_basepath = os.path.join(normalbase_path, 'diag_var_refsamples')
diag_subseq_basepath = os.path.join(faultybase_path, 'diag_subseq')
subseq_label_basepath = os.path.join(diag_subseq_basepath, 'subseq_labels')


print('ref_samples_path:\n', ref_samples_basepath)
# print('ref_var_samples_path:\n', ref_var_samples_basepath)
print('diag_subseq_path:\n', diag_subseq_basepath)

######### get paths #######################
# ref_samples_path = [os.path.join(ref_samples_basepath, x) for x in os.listdir(ref_samples_basepath)]
# ref_var_samples_path = [os.path.join(ref_var_samples_basepath, x) for x in os.listdir(ref_var_samples_basepath)]   
train_data_path = [os.path.join(train_base_path, x) for x in os.listdir(train_base_path)]


train_varlist_path = os.listdir(normalbase_path)
train_varlist_path = [os.path.join(normalbase_path, x) for x in train_varlist_path if 'varlist' in x]

######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(faultybase_path)

test_subseq_path = [os.path.join(diag_subseq_basepath, x) for x in os.listdir(diag_subseq_basepath)]
test_labels_path = [os.path.join(subseq_label_basepath, x) for x in os.listdir(subseq_label_basepath)]

# ### remove.Ds_store from all lists
train_data_path = [x for x in train_data_path if '.DS_Store' not in x]
train_varlist_path = [x for x in train_varlist_path if '.DS_Store' not in x]
varlist_path = [x for x in varlist_path if '.DS_Store' not in x]
paths_label = [x for x in paths_label if '.DS_Store' not in x]
# ref_samples_path = [x for x in ref_samples_path if '.DS_Store' not in x]
# ref_var_samples_path = [x for x in ref_var_samples_path if '.DS_Store' not in x]
test_subseq_path = [x for x in test_subseq_path if '.DS_Store' not in x if '.json' in x]
test_labels_path = [x for x in test_labels_path if '.DS_Store' not in x]


varlist_path.sort()

# print(paths_log)
# print(paths_traces)
# print(varlist_path)
# print(paths_label)

# if IS_VAR_WINDOW:
#     train_data_path = ref_var_samples_path
# else:
#     train_data_path = ref_samples_path

test_data_path = test_subseq_path

print('train_data:\n', train_data_path)
print(len(train_data_path))
print('test_data:\n', test_data_path)
print(len(test_data_path))
print('test_labels:\n', test_labels_path)



In [ ]:
############# check varlist is consistent ############
############# only for version 3 ######################

if VER == 3 or VER == 4:
    check_con, _ = is_consistent([train_varlist_path[0]]+ varlist_path) ### compare with train varlist

    if check_con != False:
        to_number = read_json(varlist_path[0])
        from_number = mapint2var(to_number)
    else:
        ### load normal varlist
        print('loading normal varlist')
        to_number = read_json(train_varlist_path[0])
        from_number = mapint2var(to_number)



In [ ]:
to_number = read_json(train_varlist_path[0])
from_number = mapint2var(to_number)

In [ ]:
# #### key finder ####
# from_number[44]

In [ ]:
############ Get variable list ######################
sorted_keys = list(from_number.keys())
sorted_keys.sort()
var_list = [from_number[key] for key in sorted_keys]   ### get the variable list
# print(var_list)

## Prepare Training Data

In [ ]:
### load all the reference samples (fixed window size)
### the ref_samples: list of list of events, list of intervals for the subseq of size 500
ref_samples = []
for train_single in train_data_path:
    ref_trace = read_traces(train_single)
    events = []
    intervals = []
    for x,y in zip(ref_trace[:-1], ref_trace[1:]):
        events.append(x[0])
        intervals.append(y[1] - x[1])
    ref_samples.append([events, intervals])

    # print(ref_samples)
    # break

In [ ]:
len(ref_samples)

In [ ]:
### TODO: 
# - deduplicate the samples

### make subseq of 50 events, with sliding interval of 1
WINDOW = 50

def prepare_data(ref_trace, WINDOW):
    ref_events = ref_trace[0]
    ref_intervals = ref_trace[1]
    # print(len(ref_events))
    # print(len(ref_intervals))
    ### we take one less event for training because we need last event as label

    X_train_event = []
    Y_train_event = []
    for i in range(len(ref_events)-WINDOW): 
        # print(i)
        # print(ref_events[i])
        # sub_seq_events.append(ref_events[i:i+WINDOW])
        # sub_seq_intervals.append(ref_intervals[i:i+WINDOW])

        ### both events and intervals are taken as input
        # _x_train = [ref_events[i:i+WINDOW], ref_intervals[i:i+WINDOW]]
        # _x_train = np.array(_x_train)
        # _x_train = np.transpose(_x_train)
        # _y_train = np.array([ref_events[i+WINDOW], ref_intervals[i+WINDOW]])

        ### only events are taken as input
        _x_train_event = [ref_events[i:i+WINDOW]]
        _y_train_event = [ref_events[i+WINDOW]]
        # _y_train_event = [ref_events[i+WINDOW], ref_intervals[i+WINDOW]]

        X_train_event.append(_x_train_event)
        Y_train_event.append(_y_train_event)

    return [X_train_event, Y_train_event]


results = pool.starmap(prepare_data, zip(ref_samples[0:], repeat(WINDOW)))
# print(results)

X_train_event = []
Y_train_event = []
for result in results:
    X_train_event.extend(result[0])
    Y_train_event.extend(result[1])
    # break
        

In [ ]:
# ### make subseq of 50 events, with sliding interval of 1
# WINDOW = 50

# sub_seq_events = []
# sub_seq_intervals = []
# X_train_event = []
# Y_train_event = []
# X_train_interval = []
# Y_train_interval = []
# for ref_trace in ref_samples[:100]:
#     # print(ref_trace)
#     ref_events = ref_trace[0]
#     ref_intervals = ref_trace[1]
#     # print(len(ref_events))
#     # print(len(ref_intervals))
#     ### we take one less event for training because we need last event as label


#     for i in range(len(ref_events)-WINDOW): 
#         # print(i)
#         # print(ref_events[i])
#         # sub_seq_events.append(ref_events[i:i+WINDOW])
#         # sub_seq_intervals.append(ref_intervals[i:i+WINDOW])

#         ### both events and intervals are taken as input
#         # _x_train = [ref_events[i:i+WINDOW], ref_intervals[i:i+WINDOW]]
#         # _x_train = np.array(_x_train)
#         # _x_train = np.transpose(_x_train)
#         # _y_train = np.array([ref_events[i+WINDOW], ref_intervals[i+WINDOW]])

#         ### only events are taken as input
#         _x_train_event = [ref_events[i:i+WINDOW]]
#         _y_train_event = [ref_events[i+WINDOW]]
#         # _y_train_event = [ref_events[i+WINDOW], ref_intervals[i+WINDOW]]

#         X_train_event.append(_x_train_event)
#         Y_train_event.append(_y_train_event)


#         # ### only intervals are taken as input
#         # _x_train_interval = [ref_intervals[i:i+WINDOW]]
#         # _y_train_interval = [ref_intervals[i+WINDOW]]
#         # X_train_interval.append(_x_train_interval)
#         # Y_train_interval.append(_y_train_interval)
        


#     # break

In [ ]:
print(np.array(X_train_event).shape)
# print(np.array(Y_train_event).shape)
# print(np.array(X_train_interval).shape)
# print(np.array(Y_train_interval).shape)

In [ ]:
### preprocess training data
X_train_event = np.array(X_train_event)
Y_train_event = np.array(Y_train_event)

X_train_event = X_train_event.reshape(X_train_event.shape[0], X_train_event.shape[2], X_train_event.shape[1])
Y_train_event = Y_train_event.reshape(Y_train_event.shape[0], Y_train_event.shape[1], 1)

### shuffle the data
X_train_event, Y_train_event = shuffle(X_train_event, Y_train_event, random_state=0)

### split the data in train, validation and test sets
X_train_event, x_test_event, Y_train_event, y_test_event = train_test_split(X_train_event, Y_train_event, test_size=0.2, random_state=0)
# X_val, X_test, Y_val, Y_test = train_test_split(X_val, Y_val, test_size=0.5, random_state=0)

print(X_train_event.shape)
print(Y_train_event.shape)

print(x_test_event.shape)
print(y_test_event.shape)

# ### preprocess training data
# X_train_interval = np.array(X_train_interval)
# Y_train_interval = np.array(Y_train_interval)

# X_train_interval = X_train_interval.reshape(X_train_interval.shape[0], X_train_interval.shape[2], X_train_interval.shape[1])
# Y_train_interval = Y_train_interval.reshape(Y_train_interval.shape[0], Y_train_interval.shape[1], 1)

# ### shuffle the data
# X_train_interval, Y_train_interval = shuffle(X_train_interval, Y_train_interval, random_state=0)

# ### split the data in train, validation and test sets
# X_train_interval, x_test_interval, Y_train_interval, y_test_interval = train_test_split(X_train_interval, Y_train_interval, test_size=0.2, random_state=0)

# print(X_train_interval.shape)
# print(Y_train_interval.shape)

# print(x_test_interval.shape)
# print(y_test_interval.shape)




In [ ]:
X_train_event.reshape(X_train_event.shape[0], -1).reshape(X_train_event.shape).shape

In [ ]:
### normalize data
from sklearn.preprocessing import MinMaxScaler
import joblib

x_test_raw = x_test_event
### check if scaler exists

if os.path.exists(f"./scalers/minmaxscaler_{CODE}_V{VER}.gz"):
    scaler = joblib.load(f"./scalers/minmaxscaler_{CODE}_V{VER}.gz")
    print('scaler loaded')

    # print(X_train_event[0])
    X_train_event = scaler.transform(X_train_event.reshape(X_train_event.shape[0], -1)).reshape(X_train_event.shape)
    x_test_event = scaler.transform(x_test_event.reshape(x_test_event.shape[0], -1)).reshape(x_test_event.shape)
    # print(X_train_event[0])
else:
    scaler = MinMaxScaler()

    # print(X_train_event[0])
    X_train_event = scaler.fit_transform(X_train_event.reshape(X_train_event.shape[0], -1)).reshape(X_train_event.shape)
    x_test_event = scaler.transform(x_test_event.reshape(x_test_event.shape[0], -1)).reshape(x_test_event.shape)
    # print(X_train_event[0])

    # print(X_train_event.shape)
    # print(x_test_event.shape)

    scaler_filename = f"./scalers/minmaxscaler_{CODE}_V{VER}.gz"
    joblib.dump(scaler, scaler_filename) 
    print('scaler saved')


In [ ]:
y_test_event[0]

In [ ]:
x_test_event[0]

## Build and Train DL Models

In [ ]:
import tensorflow as tf
import TSFEDL.models_keras as tsfedl

FINE_TUNE = False
# BACKBONE = 'forecaster'
BACKBONE = 'autoencoder'
# BACKBONE = 'rta'

if not FINE_TUNE:
    if BACKBONE == 'forecaster':
        print('training new model: forecaster')
        #### build model ####
        input = tf.keras.Input(shape=(50,1))
        model = tsfedl.HuangMeiLing(input_tensor=input, include_top=False)  ### include_top=False, to remove the top layer responsible for classification/forecast for the designed application
        print(model)
        x = model.output
        x = tf.keras.layers.LSTM(units=20)(x)
        ### Add the top module
        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(50)(x)
        # x = tf.keras.layers.Dense(10)(x)
        x = tf.keras.layers.Dense(1)(x)
        out = tf.keras.layers.Reshape([1, 1])(x)

        ### create new model
        forecaster_event = tf.keras.Model(inputs=input, outputs=out, name="forecaster")
        # forecaster_interval = tf.keras.Model(inputs=input, outputs=out, name="forecaster")

        # print(model.summary())
        # print(forecaster_event.name)
        print(forecaster_event.summary())

    elif BACKBONE == 'autoencoder':
        print('training new model: autoencoder')
        #### build model ####
        input = tf.keras.Input(shape=(50,1))
        model_list = tsfedl.YildirimOzal(input_tensor=input, include_top=False)
        ### select the autoencode model from the list
        autoencoder = model_list[0]  

        # autoencoder = tf.keras.Model(inputs=input, outputs=model.output, name="autoencoder")
        
        # print(model_list[0].summary())
        # print(model_list[1].summary())
        # print(model_list[2].summary())

        print(autoencoder.summary())
    # ### error in training, may be due to the input shape, expects multivariate input
    # elif BACKBONE == 'rta':
    #     print('training new model: rta')
    #     #### build model ####
    #     input = tf.keras.Input(shape=(50,1))
    #     model = tsfedl.YiboGao(input_tensor=input, include_top=False)
    #     # model.summary()
    #     x = model.output
    #     # x = tf.keras.layers.LSTM(units=20)(x)
    #     ### Add the top module
    #     x = tf.keras.layers.Flatten()(x)
    #     x = tf.keras.layers.Dense(50)(x)
    #     # x = tf.keras.layers.Dense(10)(x)
    #     x = tf.keras.layers.Dense(1)(x)
    #     out = tf.keras.layers.Reshape([1, 1])(x)

    #     rta_model = tf.keras.Model(inputs=input, outputs=out, name="rta")
    #     rta_model.summary()

        

else:
    if BACKBONE == 'forecaster':
        print('fine tuning model: forecaster')
        ### load the model
        model = tf.keras.models.load_model(f'./trained_models/{BACKBONE}_events_minmax_mamba2.keras')
        model.summary()
    elif BACKBONE == 'autoencoder':
        print('fine tuning model: autoencoder')
        ### load the model
        model = tf.keras.models.load_model(f'./trained_models/{BACKBONE}_events_minmax_mamba2.keras')



In [ ]:
if not FINE_TUNE:
    if BACKBONE == 'forecaster':
        model = forecaster_event
        print('Training {} model'.format(model.name))

        # print('Training {} model'.format(BACKBONE))
        ### train
        model.compile(loss='mae', optimizer='adam', metrics=['mae', 'mse'])

        # history = forecaster.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.2)
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
                # tf.keras.callbacks.ModelCheckpoint(filepath='./trained_models/tmp/model.{epoch:02d}-{loss:.2f}.keras', monitor="loss", save_best_only=True),
                tf.keras.callbacks.TensorBoard(log_dir='./logs_forecaster_event'),]
        history_event = model.fit(X_train_event, Y_train_event, epochs=50, batch_size=64, validation_split=0.2, callbacks=callbacks)

    elif BACKBONE == 'autoencoder':
        model = autoencoder
        print('Training {} model'.format(model.name))

        # print('Training {} model'.format(BACKBONE))
        ### train
        model.compile(loss='mae', optimizer='adam', metrics=['mae', 'mse'])

        # history = forecaster.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.2)
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
                # tf.keras.callbacks.ModelCheckpoint(filepath='./trained_models/tmp/model.{epoch:02d}-{loss:.2f}.keras', monitor="loss", save_best_only=True),
                tf.keras.callbacks.TensorBoard(log_dir='./logs_forecaster_event'),]
        history_event = model.fit(X_train_event, X_train_event, epochs=50, batch_size=128, validation_split=0.2, callbacks=callbacks)

    ### error in training, may be due to the input shape, expects multivariate input
    # elif BACKBONE == 'rta':
    #     model = rta_model
    #     print('Training {} model'.format(model.name))

    #     # print('Training {} model'.format(BACKBONE))
    #     ### train
    #     model.compile(loss='mae', optimizer='adam', metrics=['mae', 'mse'])

    #     # history = forecaster.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.2)
    #     callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2),
    #             tf.keras.callbacks.ModelCheckpoint(filepath='./trained_models/tmp/model.{epoch:02d}-{loss:.2f}.keras', monitor="loss", save_best_only=True),
    #             tf.keras.callbacks.TensorBoard(log_dir='./logs_forecaster_event'),]
    #     history_event = model.fit(X_train_event, Y_train_event, epochs=30, batch_size=64, validation_split=0.2, callbacks=callbacks)



else:
    if BACKBONE == 'forecaster':
        print('Fine tuning {} model'.format(model.name))
        model.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(1e-04), metrics=['mae', 'mse'])

        # history = forecaster.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.2)
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
                # tf.keras.callbacks.ModelCheckpoint(filepath='./trained_models/tmp/forecaster_event.{epoch:02d}-{loss:.2f}.keras', monitor="loss", save_best_only=True),
                tf.keras.callbacks.TensorBoard(log_dir='./logs'),]
        history_event = model.fit(X_train_event, Y_train_event , epochs=50, batch_size=128, validation_split=0.2, callbacks=callbacks)
        
    elif BACKBONE == 'autoencoder':
        print('Fine tuning {} model'.format(model.name))
        model.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(1e-04), metrics=['mae', 'mse'])

        # history = forecaster.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.2)
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
                # tf.keras.callbacks.ModelCheckpoint(filepath='./trained_models/tmp/autoencoder_event.{epoch:02d}-{loss:.2f}.keras', monitor="loss", save_best_only=True),
                tf.keras.callbacks.TensorBoard(log_dir='./logs'),]
        history_event = model.fit(X_train_event, X_train_event, epochs=50, batch_size=128, validation_split=0.2, callbacks=callbacks)

    # elif BACKBONE == 'rta':
    #     print('Fine tuning {} model'.format(model.name))
    #     model.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(1e-04), metrics=['mae', 'mse'])

    #     # history = forecaster.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.2)
    #     callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
    #             tf.keras.callbacks.ModelCheckpoint(filepath='./trained_models/tmp/rta_event.{epoch:02d}-{loss:.2f}.keras', monitor="loss", save_best_only=True),
    #             tf.keras.callbacks.TensorBoard(log_dir='./logs'),]
    #     history_event = model.fit(X_train_event, Y_train_event, epochs=30, batch_size=128, validation_split=0.2, callbacks=callbacks)




In [ ]:
import tensorflow as tf

# model = tf.keras.models.load_model('./trained_models/autoencoder_events_minmax_mamba2_V4.keras')

print('testing model {}'.format(model.name))
y_pred_event = model.predict(x_test_event)
# y_pred_interval = model.predict(x_test_interval)

print(y_pred_event.shape)
# print(y_test_interval.shape)

In [ ]:
# for i in range(10):
#     print(y_test_interval[i], y_pred_interval[i])

for i in range(10):
    # print(x_test_event[i], y_pred_event[i])
    # print(x_test_event[i].shape, y_pred_event[i].shape)
    # print(x_test_raw[i])
    descaled_x_test = scaler.inverse_transform(x_test_event[i].reshape(1, 50))
    # descaled_y_pred = scaler.inverse_transform(y_pred_event[i].reshape(1, 50))
    # print(descaled_x_test, descaled_y_pred)

    # print(descaled_x_test.shape)
    diff = y_pred_event[i] - descaled_x_test
    # print(diff.shape)
    # print(diff)
    # print(np.abs(diff))
    mae = np.mean(np.abs(diff))
    mse = np.mean(np.square(diff))
    print(mae)
    print(mse)
    # break

In [ ]:
y_test_event.shape

In [ ]:
### testing
correct = []
incorrect = []

if y_test_event.shape[1] == 2:
    for i in range(len(y_test_event)):
        yt_event = y_test_event[i][0]
        yp_event = y_pred_event[i][0]

        yt_interval = y_test_event[i][1]
        yp_interval = y_pred_event[i][1]

        # print(yt_event, yp_event)
        # print(yt_interval, yp_interval)
        yt_event = yt_event.reshape(1,)
        yp_event = yp_event.reshape(1,)
        yt_interval = yt_interval.reshape(1,)
        yp_interval = yp_interval.reshape(1,)

        if np.abs(yt_event-yp_event) < 1 and np.abs(yt_interval-yp_interval) < 20:
            # print('correct prediction')
            correct.append(y_test_event[i])
        else:
            # print('incorrect prediction')
            incorrect.append(y_test_event[i])
            # print(yt, yp)
        # break

else:
    if BACKBONE == 'forecaster':
        for i in range(len(y_test_event)):

            yt_event = y_test_event[i]
            yp_event = y_pred_event[i]

            yt_event = yt_event.reshape(1,)
            yp_event = yp_event.reshape(1,)
            print(yt_event, yp_event)
            if np.abs(yt_event-yp_event) < 1:
                # print('correct prediction')
                correct.append(y_test_event[i])
            else:
                # print('incorrect prediction')
                incorrect.append(y_test_event[i])
                # print(yt, yp)
            # break
    elif BACKBONE == 'autoencoder':
        all_mae = []
        all_mse = []
        for i in range(len(y_test_event)):

            yt_event = x_test_event[i]
            yp_event = y_pred_event[i]

            yt_event = scaler.inverse_transform(yt_event.reshape(1, 50))[0]
            yp_event = scaler.inverse_transform(yp_event.reshape(1, 50))[0]
            print(yt_event, yp_event)
            # print(yt_event.shape, yp_event.shape)
            # yt_event = [round(x) for x in yt_event[0]]
            # yp_event = [round(x) for x in yp_event[0]]
            # print(yt_event, yp_event)

            mse = 0
            mae = 0
            correct_match = True
            ######### pointwise ##########
            for x,y in zip(yt_event, yp_event):
                _mse = (x-y)**2
                _mae = np.abs(x-y)
                # print('mae:', mae)

                if not _mae < 1:
                    correct_match = False
                mse += _mse
                mae += _mae


            mse = mse/len(yt_event)
            mae = mae/len(yt_event)
            ##############################

            ########### mae>1 ############
            # mae = np.mean(np.abs(yt_event-yp_event))
            # mse = np.mean((yt_event-yp_event)**2)
            # if mae > 1:
            #     correct_match = False

            # print('mae:', mae)
            # mse = np.mean((yt_event-yp_event)**2)
            # mae = np.mean(np.abs(yt_event-yp_event))
            ##############################
            
            all_mae.append(mae)
            all_mse.append(mse)
            
            
            if correct_match:
            # if mse < 1:
                # print('correct prediction')
                correct.append(y_test_event[i])
                # print(yt_event == yp_event)
                # print([(int(x),y) for x,y in zip(yt_event, yp_event) ])
                # break
            else:
                # print('incorrect prediction')
                incorrect.append(y_test_event[i])
                # print(yt, yp)
                # print(yt_event == yp_event)
                # print([(int(x),y) for x,y in zip(yt_event, yp_event) ])
                # break
        
            # break

print('correct:', len(correct))
print('incorrect:', len(incorrect))

### accuracy
accuracy = len(correct)/len(y_test_event)
print('accuracy:', accuracy)

if BACKBONE == 'forecaster':
    ### mae
    mae = np.mean(np.abs(y_test_event-y_pred_event))
    print('mae:', mae)
    ### mse
    mse = np.mean((y_test_event-y_pred_event)**2)
    print('mse:', mse)
elif BACKBONE == 'autoencoder':
    ### mae
    mae = np.mean(all_mae)
    print('mae:', mae)
    ### mse
    mse = np.mean(all_mse)
    print('mse:', mse)

In [ ]:
y_test_event.shape

## save model

In [ ]:
if not FINE_TUNE:
    print('saving new model: {}'.format(BACKBONE))
    ### check if existing model is there
    model_name = f'./trained_models/{BACKBONE}_events_minmax_{CODE}_V{VER}.keras'
else:
    print('saving fine tuned model')
    model_name = f'./trained_models/{BACKBONE}_events_minmax_mamba+theft_V{VER}.keras'

if os.path.exists(model_name):
    print('model exists')
    raise FileExistsError
else:
    model.save(model_name)
# model.save('./trained_models/forecaster_intervals_1.keras')
# model.save_weights('./trained_models/forecaster_events.weights.h5')
